<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/train_realfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

Cloning into 'realfill'...
remote: Enumerating objects: 444, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 444 (delta 11), reused 10 (delta 7), pack-reused 425 (from 1)
Receiving objects: 100% (444/444), 20.14 MiB | 17.18 MiB/s, done.
Resolving deltas: 100% (214/214), done.
/content/realfill


In [2]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  452M  100  452M    0     0  88.6M      0  0:00:05  0:00:05 --:--:--  105M


In [3]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/jensen_images.zip -o jensen_images.zip
!unzip -q jensen_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 43.5M  100 43.5M    0     0  27.8M      0  0:00:01  0:00:01 --:--:-- 64.8M


In [4]:
%pwd
%ls -lh

total 516M
drwxr-xr-x 2 root root 4.0K Apr 26 20:02 benchmark/
-rw-r--r-- 1 root root  28K Apr 26 20:02 benchmarks.py
drwxr-xr-x 4 root root 4.0K Apr 26 20:02 data/
-rw-r--r-- 1 root root 3.4K Apr 26 20:02 infer.py
drwxr-xr-x 3 root root 4.0K Apr 22 00:01 jensen_images/
-rw-r--r-- 1 root root  44M Apr 26 20:02 jensen_images.zip
-rw-r--r-- 1 root root 1.1K Apr 26 20:02 LICENSE
-rw-r--r-- 1 root root  19K Apr 26 20:02 loftr_ranking.py
drwxr-xr-x 3 root root 4.0K Apr 26 20:02 __MACOSX/
-rw-r--r-- 1 root root 7.8K Apr 26 20:02 README.md
-rw-r--r-- 1 root root 5.9K Apr 26 20:02 README-Realfill.md
drwxr-xr-x 4 root root 4.0K May 30  2024 realfill_data_release_full/
-rw-r--r-- 1 root root 453M Apr 26 20:02 realfill_data_release_full.zip
-rw-r--r-- 1 root root  20M Apr 26 20:02 Realfill.pdf
-rw-r--r-- 1 root root  342 Apr 26 20:02 requirements-benchmarks.txt
-rw-r--r-- 1 root root  126 Apr 26 20:02 requirements.txt
-rw-r--r-- 1 root root  29K Apr 26 20:02 train_realfill.ipynb
-rw-r--r-- 1 root

In [5]:
!uv pip install -r requirements.txt --no-progress
!uv pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --no-progress

Using Python 3.11.12 environment at: /usr
Resolved 60 packages in 430ms
 Downloaded kornia-rs
 Downloaded nvidia-cuda-cupti-cu12
 Downloaded nvidia-nvjitlink-cu12
 Downloaded kornia
 Downloaded nvidia-cuda-nvrtc-cu12
 Downloaded xformers
 Downloaded nvidia-curand-cu12
 Downloaded bitsandbytes
 Downloaded nvidia-cusolver-cu12
 Downloaded nvidia-cusparse-cu12
 Downloaded nvidia-cufft-cu12
 Downloaded nvidia-cublas-cu12
 Downloaded nvidia-cudnn-cu12
Prepared 15 packages in 26.47s
Uninstalled 10 packages in 30ms
Installed 15 packages in 21ms
 + bitsandbytes==0.45.5
 + ftfy==6.3.1
 + kornia==0.8.0
 + kornia-rs==0.1.8
 - nvidia-cublas-cu12==12.5.3.2
 + nvidia-cublas-cu12==12.4.5.8
 - nvidia-cuda-cupti-cu12==12.5.82
 + nvidia-cuda-cupti-cu12==12.4.127
 - nvidia-cuda-nvrtc-cu12==12.5.82
 + nvidia-cuda-nvrtc-cu12==12.4.127
 - nvidia-cuda-runtime-cu12==12.5.82
 + nvidia-cuda-runtime-cu12==12.4.127
 - nvidia-cudnn-cu12==9.3.0.75
 + nvidia-cudnn-cu12==9.1.0.70
 - nvidia-cufft-cu12==11.2.3.61
 + nv

In [6]:
from accelerate.utils import write_basic_config

write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [ ]:
import os
from pathlib import Path

# --- Configuration Toggles ---
USE_DRIVE_STORAGE = True  # True: Use Google Drive; False: Use local Colab storage
USE_FP32 = False  # True: Use FP32 precision; False: Use FP16 (mixed)
USE_GENERATED_REF_IMAGE = False  # True: Supplement refs using prior results via LoFTR script
TARGET_TOTAL_REF_COUNT = 5  # Desired final number of reference images

# --- Base Environment Setup ---
DRIVE_BASE_DIR = "/content/drive/MyDrive/RealFill"  # Base path on Google Drive

# --- Dataset and Model Info ---
# Possible values: realfill_data_release_full, jensen_images, your_custom_dataset
os.environ["DATASET"] = "realfill_data_release_full"
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
# Possible values: RealBench, Qualitative, Custom
os.environ["BENCHMARK"] = "RealBench"
os.environ["DATASET_NUMBER"] = "22"  # Important: Set your dataset number

# --- Path to the LoFTR script ---
# Assumes loftr_ranking.py is in the current working directory of the notebook
LOFTR_SCRIPT_PATH = "loftr_ranking.py"  # Adjust if it's in a subfolder. loftr_ranking.py

print("Configuration Loaded.")

# --- Determine Base Output Prefix (Drive or Local) ---
base_output_prefix = ""
if USE_DRIVE_STORAGE:
    try:
        from google.colab import drive

        drive.mount("/content/drive", force_remount=True)
        print("Google Drive mounted.")
        base_output_prefix = (
            f'{DRIVE_BASE_DIR}/{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}'
        )
        Path(DRIVE_BASE_DIR).mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"WARNING: Drive mount/access failed ({e}). Falling back to local.")
        USE_DRIVE_STORAGE = False
        base_output_prefix = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}'
else:
    base_output_prefix = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}'
    print("Using local Colab storage.")

# --- Construct Suffixes ---
# Suffix for *this* run's output if supplementing refs
generated_suffix = "-generated" if USE_GENERATED_REF_IMAGE else ""
# Suffix based on precision setting
precision_suffix = "-fp32" if USE_FP32 else ""

# --- Define Current Run's Output Paths ---
OUTPUT_DIR = f"{base_output_prefix}-model{generated_suffix}{precision_suffix}"
OUTPUT_IMG_DIR = f"{base_output_prefix}-results{generated_suffix}{precision_suffix}"
os.environ["OUTPUT_DIR"] = OUTPUT_DIR  # For potential use by training scripts
os.environ["OUTPUT_IMG_DIR"] = OUTPUT_IMG_DIR  # For potential use by training scripts

# --- Define Dataset/Input Paths ---
# Uses Pathlib for easier handling
TRAIN_DIR = Path(
    f'{os.environ["DATASET"]}/{os.environ["BENCHMARK"]}/{os.environ["DATASET_NUMBER"]}'
)
VAL_IMG = TRAIN_DIR / "target/target.png"
VAL_MASK = TRAIN_DIR / "target/mask.png"
# REF_DIR is the TARGET directory for the LoFTR script's copy operation
REF_DIR = TRAIN_DIR / "ref"

# Set corresponding environment variables if other scripts need them
os.environ["TRAIN_DIR"] = str(TRAIN_DIR)
os.environ["VAL_IMG"] = str(VAL_IMG)
os.environ["VAL_MASK"] = str(VAL_MASK)

# --- Define SOURCE Directory for Candidate Images (Input for LoFTR script) ---
# This is the results dir from the *previous* (non-generated) run
SOURCE_RESULTS_DIR_FOR_COPY = f"{base_output_prefix}-results{precision_suffix}"

# --- Create Necessary Directories ---
# Ensure dataset, reference, and output directories exist
TRAIN_DIR.mkdir(parents=True, exist_ok=True)
REF_DIR.mkdir(parents=True, exist_ok=True)  # MUST exist before script runs
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
Path(OUTPUT_IMG_DIR).mkdir(parents=True, exist_ok=True)
# Note: The script itself checks if SOURCE_RESULTS_DIR_FOR_COPY exists

print("\n--- Paths Initialized ---")
print(f"Model Output Dir: {OUTPUT_DIR}")
print(f"Results Output Dir: {OUTPUT_IMG_DIR}")
print(f"Reference Dir (Target): {REF_DIR}")
print(f"Source Candidates Dir: {SOURCE_RESULTS_DIR_FOR_COPY}")

# Conditional Reference Image Supplementing
# If `USE_GENERATED_REF_IMAGE` is True, executes `loftr_ranking.py` to rank candidates
# from the source directory against existing references and copy the best ones.

if USE_GENERATED_REF_IMAGE:
    print("\n" + "=" * 40)
    print(f"Calling LoFTR Ranking Script: {LOFTR_SCRIPT_PATH}")
    print("=" * 40)

    # Basic checks before executing the external script
    if not os.path.exists(LOFTR_SCRIPT_PATH):
        print(f"ERROR: Cannot execute script. {LOFTR_SCRIPT_PATH} not found!")
        print("Please ensure the script is in the current directory or fix the path.")
    elif not REF_DIR.is_dir():
        print(f"ERROR: Target Reference directory '{REF_DIR}' does not exist. Skipping script.")
    else:
        # Construct the command arguments safely using quotes
        command = f"""python3 "{LOFTR_SCRIPT_PATH}" \
        --source-dir "{SOURCE_RESULTS_DIR_FOR_COPY}" \
        --ref-dir "{str(REF_DIR)}" \
        --target-count {TARGET_TOTAL_REF_COUNT}
        """
        # Optional: Add arguments like --conf-threshold 0.8 if needed
        # command += " --conf-threshold 0.8"

        print(f"Executing command:\n{command}\n")
        # Execute the command - output will be shown below
        !{command}
        print("\nLoFTR Ranking Script Execution Finished.")

    print("=" * 40)

else:
    print("\nSkipping reference image supplementing (USE_GENERATED_REF_IMAGE is False).")


# Final Environment Setup and Summary
# Sets the precision argument and prints a summary of the final configuration.

# Set precision argument for potential use in training scripts
os.environ["PRECISION_ARG"] = "--mixed_precision=fp16" if not USE_FP32 else ""

# --- Print Final Summary ---
print("\n" + "*" * 40)
print("Final Configuration Summary")
print("*" * 40)
print(f"Storage Location: {'Google Drive' if USE_DRIVE_STORAGE else 'Local Colab'}")
print(f"Supplement References Script Called: {USE_GENERATED_REF_IMAGE}")
print(f"Precision: {'FP16 (Mixed)' if not USE_FP32 else 'FP32'}")
print(f"Precision Script Arg: {os.environ.get('PRECISION_ARG', '(Default)')}")
print("-" * 10)
print(f"Model Output Dir: {os.environ['OUTPUT_DIR']}")
print(f"Results Output Dir: {os.environ['OUTPUT_IMG_DIR']}")
print("-" * 10)
print(f"Train Data Dir: {os.environ['TRAIN_DIR']}")
print(f"Reference Dir (Final State): {REF_DIR} (Verify content)")  # Remind user to check
print(f"Validation Image: {os.environ['VAL_IMG']}")
print(f"Validation Mask: {os.environ['VAL_MASK']}")
print("*" * 40)

In [ ]:
!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100 \
  --checkpointing_steps 100 \
  $PRECISION_ARG \
  --use_8bit_adam \
  --set_grads_to_none \
  --enable_xformers_memory_efficient_attention

In [ ]:
!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

In [ ]:
# Zip final inference results
!zip -r9j $OUTPUT_IMG_DIR.zip $OUTPUT_IMG_DIR
# Zip tensorboard logs
!zip -r9D $OUTPUT_DIR-tensorboard.zip $OUTPUT_DIR/logs
%ls

In [ ]:
# Install dependencies specifically for benchmarking
print("Installing benchmarking requirements...")
!uv pip install -r requirements-benchmarks.txt --no-progress
!uv pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --no-progress
print("Benchmarking requirements installed.")

Installing benchmarking requirements...
Using Python 3.11.12 environment at: /usr
Resolved 57 packages in 798ms
   Building dreamsim==0.2.1
 Downloaded open-clip-torch
      Built dreamsim==0.2.1
Prepared 3 packages in 609ms
Installed 3 packages in 3ms
 + dreamsim==0.2.1
 + lpips==0.1.4
 + open-clip-torch==2.32.0
Using Python 3.11.12 environment at: /usr
Resolved 27 packages in 720ms
Audited 27 packages in 0.18ms
Benchmarking requirements installed.


In [8]:
# Make sure you mounted your Google Drive if using it!
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

import os  # Ensure os is imported if not already

# --- Define paths for the benchmark script ---
# Ensure these variables are correctly set based on your notebook's setup cell
drive_base_dir = os.environ.get("DRIVE_BASE_DIR", "/content/drive/MyDrive/RealFill")

# --- !!! Define the correct locations for your UNZIPPED datasets !!! ---
realbench_dataset_location = (
    "/content/realfill/realfill_data_release_full"  # Default download location
)
custom_dataset_location = (
    "/content/realfill/jensen_images"  # Default download location for custom/jensen
)

benchmark_cache_dir = os.path.join(drive_base_dir, "benchmark_cache")
benchmark_report_file = os.path.join(drive_base_dir, "benchmark_report.txt")
loftr_script = "loftr_ranking.py"  # Assuming it's in the root directory

print("\n--- Running Benchmarks ---")
print(f"Using Results Base: {drive_base_dir}")
print(f"Using RealBench Dataset Dir: {realbench_dataset_location}")
print(f"Using Custom Dataset Dir: {custom_dataset_location}")
print(f"Using Cache Dir: {benchmark_cache_dir}")
print(f"Saving Report To: {benchmark_report_file}")
print("-" * 30)

# Construct the command, adding the new dataset directory arguments
benchmark_command = f"""python benchmarks.py \\
    --results_base_dir "{drive_base_dir}" \\
    --realbench_dataset_dir "{realbench_dataset_location}" \\
    --custom_dataset_dir "{custom_dataset_location}" \\
    --cache_dir "{benchmark_cache_dir}" \\
    --output_file "{benchmark_report_file}" \\
    --loftr_script_path "{loftr_script}"
"""
# Optional: Add --force_recalc all or --metrics PSNR SSIM etc.
# benchmark_command += " --force_recalc all"

print(f"Executing command:\n{benchmark_command}\n")

# Execute the benchmark script
!{benchmark_command}

print("\n--- Benchmarking Script Finished ---")
print(f"Report saved to: {benchmark_report_file}")

Mounted at /content/drive

--- Running Benchmarks ---
Using Results Base: /content/drive/MyDrive/RealFill
Using RealBench Dataset Dir: /content/realfill/realfill_data_release_full
Using Custom Dataset Dir: /content/realfill/jensen_images
Using Cache Dir: /content/drive/MyDrive/RealFill/benchmark_cache
Saving Report To: /content/drive/MyDrive/RealFill/benchmark_report.txt
------------------------------
Executing command:
python benchmarks.py \
    --results_base_dir "/content/drive/MyDrive/RealFill" \
    --realbench_dataset_dir "/content/realfill/realfill_data_release_full" \
    --custom_dataset_dir "/content/realfill/jensen_images" \
    --cache_dir "/content/drive/MyDrive/RealFill/benchmark_cache" \
    --output_file "/content/drive/MyDrive/RealFill/benchmark_report.txt" \
    --loftr_script_path "loftr_ranking.py"


Benchmark Runner Initialized.
Results Base Dir: /content/drive/MyDrive/RealFill
Dataset Directories Map: {'RealBench': PosixPath('/content/realfill/realfill_data_releas